In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/05 14:49:13 WARN Utils: Your hostname, mx resolves to a loopback address: 127.0.0.1; using 192.168.15.13 instead (on interface eth0)
23/03/05 14:49:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/05 14:49:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-06.csv.gz')

In [4]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [16]:
from pyspark.sql import types

schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True),
])

In [28]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-06.csv.gz')

In [18]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [19]:
#df = df.repartition(12)

In [20]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02510|2021-06-25 12:58:46|2021-06-25 13:28:10|          74|         161|      N|                  null|
|              B02765|2021-06-12 13:07:29|2021-06-12 13:13:08|         198|         198|      N|                B02765|
|              B02510|2021-06-05 15:06:03|2021-06-05 15:13:43|         258|          76|      N|                  null|
|              B02879|2021-06-23 11:21:46|2021-06-23 11:32:17|         161|          68|      N|                B02879|
|              B02872|2021-06-16 20:58:43|2021-06-16 21:04:34|         144|         211|      N|                B02872|
|              B02510|2021-06-12 14:52:1

In [ ]:
# df.write.parquet('fhvhv_hw/')

In [26]:
df.summary().show()

+-------+--------------------+------------------+------------------+--------+----------------------+
|summary|dispatching_base_num|      PULocationID|      DOLocationID| SR_Flag|Affiliated_base_number|
+-------+--------------------+------------------+------------------+--------+----------------------+
|  count|            14961892|          14961892|          14961892|14961892|              10751439|
|   mean|                null|137.17778660613243|141.03749793141134|    null|                  null|
| stddev|                null| 75.97052669218735| 78.42338911468418|    null|                  null|
|    min|              B02395|                 1|                 1|       N|                B00446|
|    25%|                null|                72|                74|    null|                  null|
|    50%|                null|               138|               141|    null|                  null|
|    75%|                null|               210|               216|    null|              

In [29]:
df.createOrReplaceTempView('homework_table')

In [32]:
#Question 3: How many taxi trips were started on June 15th? *

spark.sql('''
select
    count(*) as trips_june_15
from 
    homework_table
where
    day(pickup_datetime) = 15             
          
          
'''
).show()

+-------------+
|trips_june_15|
+-------------+
|       452470|
+-------------+



In [38]:
#Question 4: How long is the longest trip in the dataset? 

spark.sql('''
select
    datediff(dropoff_datetime, pickup_datetime) as trips_longest,
    pickup_datetime,
    dropoff_datetime
from 
    homework_table
order by
    trips_longest
    desc                
'''
).show()

+-------------+-------------------+-------------------+
|trips_longest|    pickup_datetime|   dropoff_datetime|
+-------------+-------------------+-------------------+
|            3|2021-06-25 13:55:41|2021-06-28 08:48:25|
|            1|2021-06-01 20:34:55|2021-06-02 01:50:39|
|            1|2021-06-01 21:43:31|2021-06-02 03:38:23|
|            1|2021-06-01 22:52:08|2021-06-02 04:49:02|
|            1|2021-06-01 22:56:03|2021-06-02 00:03:13|
|            1|2021-06-01 21:28:17|2021-06-02 05:09:22|
|            1|2021-06-01 21:45:51|2021-06-02 01:00:06|
|            1|2021-06-01 22:52:57|2021-06-02 00:04:18|
|            1|2021-06-01 22:29:33|2021-06-02 00:16:49|
|            1|2021-06-01 22:57:44|2021-06-02 00:04:09|
|            1|2021-06-01 22:52:40|2021-06-02 00:13:17|
|            1|2021-06-01 22:12:20|2021-06-02 01:19:52|
|            1|2021-06-01 22:57:22|2021-06-02 00:07:05|
|            1|2021-06-01 22:46:58|2021-06-02 00:34:20|
|            1|2021-06-01 22:55:06|2021-06-02 02

In [47]:
df_zones = spark.read.csv('taxi_zone_lookup.csv', header=True)

In [48]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [49]:
df_zones.registerTempTable('zones')

/usr/local/lib/python3.9/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [55]:
# Question 6: What is the name of the most frequent pickup location zone?

spark.sql('''
select
    count(*) as freq_pu_location,
    hw.PULocationID,
    zo.Zone
from 
    homework_table as hw
left join
    zones zo on hw.PULocationID = zo.LocationID
group by
    hw.PULocationID,
    zo.Zone
order by
    freq_pu_location
    desc             
'''
).show()

+----------------+------------+--------------------+
|freq_pu_location|PULocationID|                Zone|
+----------------+------------+--------------------+
|          231279|          61| Crown Heights North|
|          221244|          79|        East Village|
|          188867|         132|         JFK Airport|
|          187929|          37|      Bushwick South|
|          186780|          76|       East New York|
|          164344|         231|TriBeCa/Civic Center|
|          161596|         138|   LaGuardia Airport|
|          158937|         234|            Union Sq|
|          154698|         249|        West Village|
|          152493|           7|             Astoria|
|          151020|         148|     Lower East Side|
|          147673|          68|        East Chelsea|
|          146402|          42|Central Harlem North|
|          143683|         255|Williamsburg (Nor...|
|          143594|         181|          Park Slope|
|          141427|         225|  Stuyvesant He